# 工作总结

## 1. 定义绳长到曲率的映射函数
根据分段恒定曲率 (PCC) 假设，从三根对称分布的缆绳长度变化量 ($dl_1$, $dl_2$, $dl_3$) 推导出曲率 ($u_x$, $u_y$)。这正是 v2 和 v3 版本计算公式背后的几何原理。

### 1. 核心假设
* **PCC 模型**: 假设机器人的一小段是按照一个恒定的曲率弯曲的，就像一个圆弧。我们用曲率向量 $u$ 来描述这个弯曲。
* **几何设置**:
    * 考虑机器人的一段，其中心骨干的初始（或参考）长度为 $L$ (在代码中对应 $L0_{seg}$ 或 $L_{ref}$)。
    * 有三根缆绳平行于中心骨干，距离中心骨干的径向距离都是 $d$。
* **对称布局**: 我们需要假设一个缆绳的布局。一个标准且常用的对称布局是：在垂直于骨干的横截面上建立一个局部坐标系 (x, y)，z 轴沿骨干切线方向。
    * 缆绳 1 位于 $\theta_1=0$ 度 (例如，在 +x 轴上)。
    * 缆绳 2 位于 $\theta_2=2\pi/3$ (120 度)。
    * 缆绳 3 位于 $\theta_3=4\pi/3$ (240 度)。
    * 因此，第 $i$ 根缆绳相对于中心骨干的位置向量是 $d_i=[d\cos\theta_i,d\sin\theta_i,0]$。
* **曲率向量定义**:
    * 我们定义 $u_x$ 为绕 y 轴的曲率， $u_y$ 为绕 x 轴的曲率。
    * 假设没有扭转 ($u_z=0$)。
    * 为了匹配你代码中 ODE 类的 `u_hat` 矩阵 `[[0, 0, uy], [0, 0, -ux], [-uy, ux, 0]]` (该矩阵用于计算 $R'=R \cdot u_{hat}$)，曲率向量需要定义为 $u=[-u_y,u_x,0]$。因为 $R \cdot [u]_{\times}$ 中，$u_x$ 分量会乘以 $d_y$ 分量贡献给长度变化，而 $u_y$ 分量会乘以 $d_x$ 分量。
* **缆绳长度与骨干变形的关系**: 当中心骨干以曲率 $u$ 弯曲，并且可能伴有轴向应变 $\epsilon$ 时，距离中心 $d_i$ 处的缆绳 $i$ 的总长度 $l_i$ 近似为：
    $$l_i \approx \int_0^L (1+\epsilon-d_i(s) \cdot u(s))ds$$
    在 PCC 假设下（$u$ 恒定， $\epsilon$ 也常假设为恒定或用平均值代替），可以简化为：
    $$l_i \approx L(1+\epsilon-d_i \cdot u)$$
    其中 $L$ 是当前骨干的长度。
* **长度变化量 $dl_i$**: 我们更关心的是长度变化量 $dl_i=l_i-L_0$ (假设初始时所有缆绳长度都等于骨干初始长度 $L_0$)。
    $$dl_i=L(1+\epsilon-d_i \cdot u)-L_0$$
    令 $L \approx L_0$ (或者用 $L_{ref}$ 代替 $L$) 并且令 $L_e=L(1+\epsilon)-L_0$ 代表纯粹由轴向伸缩（包括应变）引起的长度变化（近似等于 $avg_{dl}$），则上式简化为：
    $$dl_i \approx L_e-L \cdot (d_i \cdot u)$$
    这里的 $L$ 就是计算时使用的参考长度 ($L0_{seg}$ 或 $L_{current\_estimate}$)。

### 2. 推导步骤
* **计算缆绳位置向量 $d_i$**:
    * $d_1=[d\cos(0),d\sin(0),0]=[d,0,0]$
    * $d_2=[d\cos(2\pi/3),d\sin(2\pi/3),0]=[-d/2,d\sqrt{3}/2,0]$
    * $d_3=[d\cos(4\pi/3),d\sin(4\pi/3),0]=[-d/2,-d\sqrt{3}/2,0]$
* **计算点积 $d_i \cdot u$**: 使用 $u=[-u_y,u_x,0]$
    * $d_1 \cdot u = [d,0,0] \cdot [-u_y,u_x,0] = -du_y$
    * $d_2 \cdot u = [-d/2,d\sqrt{3}/2,0] \cdot [-u_y,u_x,0] = (-d/2)(-u_y)+(d\sqrt{3}/2)(u_x) = \frac{d}{2}u_y+\frac{d\sqrt{3}}{2}u_x$
    * $d_3 \cdot u = [-d/2,-d\sqrt{3}/2,0] \cdot [-u_y,u_x,0] = (-d/2)(-u_y)+(-d\sqrt{3}/2)(u_x) = \frac{d}{2}u_y-\frac{d\sqrt{3}}{2}u_x$
* **建立长度变化方程组**: 使用 $dl_i=L_e-L \cdot (d_i \cdot u)$ (令 $L$ 为参考长度)
    * $dl_1 = L_e-L(-du_y) = L_e+Ldu_y$
    * $dl_2 = L_e-L(\frac{d}{2}u_y+\frac{d\sqrt{3}}{2}u_x)$
    * $dl_3 = L_e-L(\frac{d}{2}u_y-\frac{d\sqrt{3}}{2}u_x)$
* **求解 $u_x, u_y, L_e$**: 这是一个关于三个未知数 ($u_x, u_y, L_e$) 的三元线性方程组。
    * **求解 $L_e$**: 将三个方程相加：
        $$dl_1+dl_2+dl_3=(L_e+Ldu_y)+(L_e-L\frac{d}{2}u_y-L\frac{d\sqrt{3}}{2}u_x)+(L_e-L\frac{d}{2}u_y+L\frac{d\sqrt{3}}{2}u_x)$$       $$dl_1+dl_2+dl_3=3L_e$$       $$L_e = \frac{dl_1+dl_2+dl_3}{3} = avg_{dl}$$
        这说明平均缆绳长度变化量等于纯轴向伸缩量 $L_e$。
    * **求解 $u_x$**: 将方程 (3) 减去方程 (2)：
        $$dl_3-dl_2=(L_e-L\frac{d}{2}u_y+L\frac{d\sqrt{3}}{2}u_x)-(L_e-L\frac{d}{2}u_y-L\frac{d\sqrt{3}}{2}u_x)$$       $$dl_3-dl_2=Ld\sqrt{3}u_x$$       $$u_x = \frac{dl_3-dl_2}{L \cdot d \cdot \sqrt{3}}$$
    * **求解 $u_y$**: 从方程 (1) 可以得到 $Ldu_y=dl_1-L_e$ 。将 $L_e=avg_{dl}$ 代入：
        $$Ldu_y=dl_1-\frac{dl_1+dl_2+dl_3}{3}=\frac{3dl_1-dl_1-dl_2-dl_3}{3}=\frac{2dl_1-dl_2-dl_3}{3}$$       $$u_y = \frac{2dl_1-dl_2-dl_3}{3 \cdot L \cdot d}$$

### 3. 最终公式
我们就得到了之前在 v2 和 v3 中使用的公式：
$$u_x = \frac{dl_3-dl_2}{L \cdot d \cdot \sqrt{3}}$$
$$u_y = \frac{2 \cdot dl_1-dl_2-dl_3}{3 \cdot L \cdot d}$$

其中 $L$ 是所使用的参考长度（在 v2 中是 $L0_{seg}$，在 v3 中是 $L_{current}$）。
这个推导明确地展示了在 PCC 和对称假设下，两个方向的曲率是如何由所有三个缆绳的长度变化通过特定的线性组合决定的。系数（如 2, -1, -1, $\sqrt{3}$, 3）直接来源于所假设的几何布局。

In [ ]:
def calculate_curvatures_from_dl_v4(dl_segment, d, L0_seg, AXIAL_ACTION_SCALE=1.0):

    ux_calc = 0.0
    uy_calc = 0.0

    if abs(d) < 1e-9:
        print("[警告] calculate_curvatures_from_dl_v3: 缆绳距离 'd' 接近于零。")
        return ux_calc, uy_calc

    # 物理缆绳的长度变化
    dl_l1 = dl_segment[0] # 物理 l1 @ 150 deg
    dl_l2 = dl_segment[1] # 物理 l2 @ 270 deg
    dl_l3 = dl_segment[2] # 物理 l3 @ 30 deg

    # 映射到标准0,120,240度公式的输入
    # Standard Cable 1 (0 deg) -> effective_dl_c1
    # Standard Cable 2 (120 deg) -> effective_dl_c2
    # Standard Cable 3 (240 deg) -> effective_dl_c3
    effective_dl_c1 = dl_l3  # 物理 l3 (30 deg)
    effective_dl_c2 = dl_l1  # 物理 l1 (150 deg)
    effective_dl_c3 = dl_l2  # 物理 l2 (270 deg)


    avg_dl_effective = (effective_dl_c1 + effective_dl_c2 + effective_dl_c3) / 3.0
    avg_dl_physical = (dl_l1 + dl_l2 + dl_l3) / 3.0
    L_current_estimate = L0_seg + avg_dl_physical * AXIAL_ACTION_SCALE

    if abs(L_current_estimate) < 1e-9:
        print(f"[警告] calculate_curvatures_from_dl_v3: 当前估计长度 'L_current_estimate' ({L_current_estimate:.4f}) 接近于零。")
        return 0.0, 0.0

    Ld = L_current_estimate * d

    ux_denominator = Ld * math.sqrt(3.0)
    if abs(ux_denominator) > 1e-9:
        ux_calc = (effective_dl_c3 - effective_dl_c2) / ux_denominator
        # ux_calc = (dl_l2 - dl_l1) / ux_denominator
    else:
        ux_calc = 0.0

    uy_denominator = 3.0 * Ld
    if abs(uy_denominator) > 1e-9:
        uy_calc = (2.0 * effective_dl_c1 - effective_dl_c2 - effective_dl_c3) / uy_denominator
        # uy_calc = (2.0 * dl_l3 - dl_l1 - dl_l2) / uy_denominator
    else:
        uy_calc = 0.0
    return ux_calc, uy_calc

## 2. 新增绳长变化长度对机器人曲率的影响

由于机械臂在被线驱动时，机械臂本身的长度会随着绳长的改变而伸缩，所以如果还是在原来的机械臂长度来进行形态的计算时会产生误差。所以新增计算机械臂长度的变化函数。

根据三根绳的绳长变化，也就是 $dl_1$， $dl_2$ 与 $dl_3$ 相加到一起，计算三根绳长的总变化长度。再将得到的结果除以3，得到的数值就当作机械臂长度的变化量 $\Delta L_{arm}$：

$$\Delta L_{arm} = \frac{dl_1 + dl_2 + dl_3}{3}$$

接下来算机械臂的下一时刻形态时，将采用此更新后的机械臂长度（例如，$L_{next} = L_{current} + \Delta L_{arm}$）进行计算。

In [ ]:
ux, uy = utils.calculate_curvatures_from_dl_v3(dl_segment, cable_distance, L0_seg, AXIAL_ACTION_SCALE)
avg_dl = np.mean(dl_segment)

## 3. 新增轴向压缩（应变）的考量

当机械臂向特定方向弯曲时，其自身的有效长度可能会发生变化。为了更精确地描述这种现象，我们引入轴向应变（$\epsilon$）的概念。这里采用一个简化的经验模型，该模型假设机械臂的弯曲会导致其轴向上的压缩。

在此模型中，轴向应变被认为与总曲率的平方成正比。首先，计算总曲率的平方，该值等于x方向曲率 $u_x$ 的平方与y方向曲率 $u_y$ 的平方之和，即 $u_x^2 + u_y^2$。

随后，根据以下关系式计算轴向应变 $\epsilon$ 的大小：
$$\epsilon = k_{strain} \cdot (u_x^2 + u_y^2)$$
或者，在某些实现中，可能会考虑乘以初始段长 $L_0$ 来得到长度变化量，但通常 $\epsilon$ 本身代表无量纲的应变。

由于这是一个经验公式，其中的尺度因子 $k_{strain}$ 没有统一的理论值。该参数非常关键，需要根据具体的机械臂结构和实验数据进行仔细调整，以确保模型的准确性。

计算得到的轴向应变（或由此产生的压缩效应）将会被用于后续的常微分方程（ODE）求解过程中，以确定机械臂的新形态。如果计算出的轴向应变 $\epsilon$ 为负值（表示压缩），那么在基于Cosserat杆理论进行求解时，模型中每个微小段的积分长度会相应减小。这将导致最终计算出的机械臂整体形态在长度上变短，从而实现了对轴向压缩效果的模拟。

In [ ]:
    def _calculate_axial_strain(self):
        curvature_squared = self.ux**2 + self.uy**2
        calculated_epsilon = self.k_strain * curvature_squared * self.l0 
        return calculated_epsilon